In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import zscore

# Import necessary libraries for model selection
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import (
    StackingRegressor,
    BaggingRegressor,
    AdaBoostRegressor,
    VotingRegressor,
)
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import warnings


#### Seleção do modelo para prever a colunas Approx. Total Revenue(INR)
def treat_outliers(df, col):
    z_scores = zscore(df[col])
    df_no_outliers = df[(z_scores < 3)]
    return df_no_outliers


def training_total_revenue(df):
    print("Training model to predict Approx. Total Revenue(INR)...")

    # Tratamento de outliers
    df = treat_outliers(df, "Approx. Total Revenue(INR)")

    # Preparando os dados para modelagem
    X = df.drop(
        columns=[
            "Approx. Total Revenue(INR)",
            "Quantity Sold (liters/kg)",
            "Quantity in Stock (liters/kg)",
            "Total Value",
        ]
    )
    y = df["Approx. Total Revenue(INR)"]

    # Dividindo em treino e teste
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # Definindo colunas categóricas e numéricas
    categorical_cols = [
        "Location",
        "Farm Size",
        "Product Name",
        "Brand",
        "Customer Location",
        "Sales Channel",
    ]
    numerical_cols = [
        "Total Land Area (acres)",
        "Price per Unit (sold)",

        "Number of Cows",
        "Quantity (liters/kg)",
        "Price per Unit",
        # "Total Value",
        "Shelf Life (days)",
        # "Quantity in Stock (liters/kg)",
        "Minimum Stock Threshold (liters/kg)",
        "Reorder Quantity (liters/kg)",
        "Days Before Expire",
        "Days to Sell",
    ]

    # Pré-processamento
    cat_preprocessor = Pipeline([("onehot", OneHotEncoder(handle_unknown="ignore"))])
    num_preprocessor = Pipeline([("scaler", StandardScaler())])

    preprocessor = ColumnTransformer(
        [
            ("cat", cat_preprocessor, categorical_cols),
            ("num", num_preprocessor, numerical_cols),
        ]
    )

    # Pré-processando os dados
    X_train_preprocessed = preprocessor.fit_transform(X_train)
    X_test_preprocessed = preprocessor.transform(X_test)

    # Normalizando o target
    scaler = StandardScaler()
    y_train_normalized = scaler.fit_transform(y_train.values.reshape(-1, 1))
    y_test_normalized = scaler.transform(y_test.values.reshape(-1, 1))

    # Definindo e ajustando o modelo Decision Tree
    decision_tree = DecisionTreeRegressor(random_state=42)

    # Otimização de hiperparâmetros
    decision_tree_params = {
        "max_depth": [None, 5, 10, 15],
        "min_samples_split": [2, 5, 10],
        "min_samples_leaf": [1, 2, 4],
    }

    print("Tuning hyperparameters for Decision Tree...")
    grid_search = GridSearchCV(
        decision_tree,
        decision_tree_params,
        cv=5,
        scoring="neg_mean_squared_error",
        n_jobs=-1,
    )
    grid_search.fit(X_train_preprocessed, y_train_normalized.ravel())
    RMSE = (-1 * grid_search.best_score_) ** 0.5
    best_decision_tree = grid_search.best_estimator_
    print(f"Best Parameters: {grid_search.best_params_}")
    print(f"Best Mean MSE: {-1 * grid_search.best_score_:.4f}\n")
    print(f"Best Mean RMSE: {RMSE}\n")


    # Avaliação final com cross-validation
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(
        best_decision_tree,
        X_train_preprocessed,
        y_train_normalized.ravel(),
        cv=cv,
        scoring="neg_mean_squared_error",
    )
    mean_mse = np.mean((-1) * scores)
    print(f"Decision Tree - Final Mean MSE: {mean_mse:.4f}")

    # Análise de importância de features
    categorical_transformer = preprocessor.named_transformers_["cat"]
    categorical_feature_names = categorical_transformer.named_steps[
        "onehot"
    ].get_feature_names_out(input_features=categorical_cols)
    feature_names = list(categorical_feature_names) + numerical_cols

    if hasattr(best_decision_tree, "feature_importances_"):
        feature_importances = best_decision_tree.feature_importances_
        sorted_indices = np.argsort(feature_importances)[::-1]

        print("\nFeature Importances for Decision Tree:")
        for i in sorted_indices:
            print(f"{feature_names[i]}: {feature_importances[i]:.4f}")
    else:
        print("The Decision Tree model does not have feature importances.")

    # Retornando os objetos necessários
    total_rev = {
        "model": best_decision_tree,
        "preprocessor": preprocessor,
        "scaler": scaler,
        "feature_names": feature_names,
    }
    return total_rev

def training_total_quantity_sold(df):
    print("Training model to predict Quantity Sold (liters/kg)...")
    # Tratamento de outliers
    df = treat_outliers(df, "Approx. Total Revenue(INR)")

    # Preparando os dados para modelagem
    X = df.drop(
        columns=[
            "Quantity Sold (liters/kg)",
            "Total Value",
            "Quantity in Stock (liters/kg)",

        ]
    )
    y = df["Quantity Sold (liters/kg)"]

    # Dividindo em treino e teste
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # Definindo colunas categóricas e numéricas
    categorical_cols = [
        "Location",
        "Farm Size",
        "Product Name",
        "Brand",
        "Customer Location",
        "Sales Channel",
    ]

    numerical_cols = [
        "Total Land Area (acres)",
        "Number of Cows",
        "Quantity (liters/kg)",
        "Price per Unit",
        "Price per Unit (sold)",
        "Approx. Total Revenue(INR)",

        # "Total Value",
        "Shelf Life (days)",
        "Minimum Stock Threshold (liters/kg)",
        "Reorder Quantity (liters/kg)",
        "Days Before Expire",
        "Days to Sell",
    ]
    
    

    # Pré-processamento
    cat_preprocessor = Pipeline([("onehot", OneHotEncoder(handle_unknown="ignore"))])
    num_preprocessor = Pipeline([("scaler", StandardScaler())])

    preprocessor = ColumnTransformer(
        [
            ("cat", cat_preprocessor, categorical_cols),
            ("num", num_preprocessor, numerical_cols),
        ]
    )

    # Pré-processando os dados
    X_train_preprocessed = preprocessor.fit_transform(X_train)
    X_test_preprocessed = preprocessor.transform(X_test)

    # Normalizando o target
    scaler = StandardScaler()
    y_train_normalized = scaler.fit_transform(y_train.values.reshape(-1, 1))
    y_test_normalized = scaler.transform(y_test.values.reshape(-1, 1))

    # Definindo e ajustando o modelo Decision Tree
    decision_tree = DecisionTreeRegressor(random_state=42)

    # Otimização de hiperparâmetros
    decision_tree_params = {
        "max_depth": [None, 5, 10, 15],
        "min_samples_split": [2, 5, 10],
        "min_samples_leaf": [1, 2, 4],
    }

    print("Tuning hyperparameters for Decision Tree...")
    grid_search = GridSearchCV(
        decision_tree,
        decision_tree_params,
        cv=5,
        scoring="neg_mean_squared_error",
        n_jobs=-1,
    )
    grid_search.fit(X_train_preprocessed, y_train_normalized.ravel())
    
    RMSE = (-1 * grid_search.best_score_) ** 0.5
    best_decision_tree = grid_search.best_estimator_
    print(f"Best Parameters: {grid_search.best_params_}")
    print(f"Best Mean MSE: {-1 * grid_search.best_score_:.4f}\n")
    print(f"Best Mean RMSE: {RMSE}\n")

    # Avaliação final com cross-validation
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(
        best_decision_tree,
        X_train_preprocessed,
        y_train_normalized.ravel(),
        cv=cv,
        scoring="neg_mean_squared_error",
    )
    mean_mse = np.mean((-1) * scores)
    print(f"Decision Tree - Final Mean MSE: {mean_mse:.4f}")

    # Análise de importância de features
    categorical_transformer = preprocessor.named_transformers_["cat"]
    categorical_feature_names = categorical_transformer.named_steps[
        "onehot"
    ].get_feature_names_out(input_features=categorical_cols)
    feature_names = list(categorical_feature_names) + numerical_cols

    if hasattr(best_decision_tree, "feature_importances_"):
        feature_importances = best_decision_tree.feature_importances_
        sorted_indices = np.argsort(feature_importances)[::-1]

        print("\nFeature Importances for Decision Tree:")
        for i in sorted_indices:
            print(f"{feature_names[i]}: {feature_importances[i]:.4f}")
    else:
        print("The Decision Tree model does not have feature importances.")

    # Retornando os objetos necessários
    quant_sold = {
        "model": best_decision_tree,
        "preprocessor": preprocessor,
        "scaler": scaler,
        "feature_names": feature_names,
    }
    return quant_sold


dados_path = r"C:\Users\alexa\OneDrive\Anexos\Fiap\projeto_fase4\Enterprise Challenge\data_analysis\data\dados.csv"
df = pd.read_csv(dados_path)
total_rev = training_total_revenue(df)
quant_sold = training_total_quantity_sold(df)


Training model to predict Approx. Total Revenue(INR)...
Tuning hyperparameters for Decision Tree...
Best Parameters: {'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2}
Best Mean MSE: 0.5361

Best Mean RMSE: 0.7321780934613439

Decision Tree - Final Mean MSE: 0.5415

Feature Importances for Decision Tree:
Quantity (liters/kg): 0.5189
Price per Unit (sold): 0.4218
Price per Unit: 0.0451
Days to Sell: 0.0082
Location_West Bengal: 0.0051
Customer Location_Haryana: 0.0009
Days Before Expire: 0.0000
Reorder Quantity (liters/kg): 0.0000
Minimum Stock Threshold (liters/kg): 0.0000
Total Land Area (acres): 0.0000
Number of Cows: 0.0000
Sales Channel_Retail: 0.0000
Sales Channel_Online: 0.0000
Customer Location_West Bengal: 0.0000
Customer Location_Uttar Pradesh: 0.0000
Customer Location_Telangana: 0.0000
Customer Location_Tamil Nadu: 0.0000
Customer Location_Rajasthan: 0.0000
Shelf Life (days): 0.0000
Customer Location_Maharashtra: 0.0000
Customer Location_Madhya Pradesh: 0.0000
Cu

In [84]:
def predict(machine_leaning, new_data_df):
    new_data_preprocessed = machine_leaning['preprocessor'].transform(new_data_df)

# Make predictions using the trained model
    predictions = machine_leaning['model'].predict(new_data_preprocessed)

    # If the target variable was normalized, reverse the normalization
    predictions_original_scale = machine_leaning['scaler'].inverse_transform(predictions.reshape(-1, 1))

    prediction = predictions_original_scale.flatten()
    return prediction

In [66]:
df.head(5)

,Unnamed: 0,Location,Total Land Area (acres),Number of Cows,Farm Size,Date,Product ID,Product Name,Brand,Quantity (liters/kg),...,Quantity Sold (liters/kg),Price per Unit (sold),Approx. Total Revenue(INR),Customer Location,Sales Channel,Quantity in Stock (liters/kg),Minimum Stock Threshold (liters/kg),Reorder Quantity (liters/kg),Days Before Expire,Days to Sell
0,0,Telangana,310.84,96,Medium,2022-01-11,5,Ice Cream,Dodla Dairy,222.40,...,7,82.24,575.68,Madhya Pradesh,Wholesale,215,19.55,64.03,10,15
1,1,Uttar Pradesh,19.19,44,Large,2021-10-12,1,Milk,Amul,687.48,...,558,39.24,21895.92,Kerala,Wholesale,129,43.17,181.10,13,9
2,2,Tamil Nadu,581.69,24,Medium,2022-02-03,4,Yogurt,Dodla Dairy,503.48,...,256,33.81,8655.36,Madhya Pradesh,Online,247,15.10,140.83,10,20
3,3,Telangana,908.00,89,Small,2019-07-01,3,Cheese,Britannia Industries,823.36,...,601,28.92,17380.92,Rajasthan,Online,222,74.50,57.68,25,47
4,4,Maharashtra,861.95,21,Medium,2020-10-17,8,Buttermilk,Mother Dairy,147.77,...,145,83.07,12045.15,Jharkhand,Retail,2,76.02,33.40,11,0


In [91]:
# Verificar correlação entre features
price_per_unit = 10
price_per_unit_sold = 12
quantity_liters_kg = 830.40
total_value = price_per_unit * quantity_liters_kg

new_data_dict = {
    'Location': ['Delhi'],
    'Farm Size': ['Medium'],
    'Product Name': ['Cheese'],
    'Brand': ['Amul'],
    'Quantity in Stock (liters/kg)': [100],
    'Customer Location': ['Delhi'],
    'Sales Channel': ['Retail'],
    'Total Land Area (acres)': [310],
    'Number of Cows': [96],
    'Quantity (liters/kg)': [quantity_liters_kg],
    'Price per Unit': [price_per_unit],
    'Price per Unit (sold)': [price_per_unit_sold],
    # 'Total Value': [total_value],
    'Shelf Life (days)': [10],
    'Minimum Stock Threshold (liters/kg)': [50],
    'Reorder Quantity (liters/kg)': [19.55],
    'Days Before Expire': [20],
    'Days to Sell': [30],
}

new_data_df = pd.DataFrame(new_data_dict)


In [93]:
total_revenue = predict(total_rev, new_data_df)
new_data_df['Approx. Total Revenue(INR)'] = total_rev
quantity_sold = predict(quant_sold, new_data_df)
print('Total Rev:', total_rev)
print('Quantity Sold Predicted:', quantity_sold)


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices